first we import all whats needed

In [1]:
import cv2                                  # for video processing and drawing
import numpy as np                          # for all numerical operations
from collections import OrderedDict         # Keeps track of objects in the order they were added
from scipy.spatial import distance as dist  # Computes pairwise distances between points (used for tracking)
from IPython.display import Video, HTML, display
import os

Here the needed funktions shall be added

In [2]:

def nms_boxes(boxes, scores, iou_thresh=0.30):
    """Simple Non-Maximum Suppression for [x1,y1,x2,y2] boxes."""
    boxes = np.array(boxes, dtype=float)
    scores = np.array(scores, dtype=float)
    if len(boxes) == 0:
        return []

    x1, y1, x2, y2 = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]  # high score first
    keep = []

    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter)
        # nur Boxen behalten, die wenig Überschneidung haben
        inds = np.where(iou <= iou_thresh)[0]
        order = order[inds + 1]
    return keep


now all needed pathes were added

In [3]:
video_path       = "../videos/CarsOnAHighway_30fps.mp4"        # the video we do use
prototxt_path    = "../models/deploy.prototxt"                 # the network architecture of MobileNetSSD 
model_path       = "../models/mobilenet_iter_73000.caffemodel" # the trained weights of MobileNetSSD 
output_path      = "../output_tracked.mp4"                     # the video we do want to create
output_path_h264 = "../output_tracked_h264.mp4"             # the video we do want to create with h264 codec

check if the file was executed already, if so delete all craeted files

In [4]:

files_to_delete = [output_path, output_path_h264]

for file in files_to_delete:
    if os.path.exists(file):
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")
    else:
        print(f"Not found: {file}")


Deleted: ../output_tracked.mp4
Deleted: ../output_tracked_h264.mp4


Load MobileNetSSD - what happens here is that the architecture information and the calculated weights are taken and a model is created out of that.
A list is created with all detectable opbjects to have an ID <-> name relation later

In [5]:
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

now the improved centraid tracker is created

In [6]:
# Improved Centroid Tracker
class CentroidTracker:
    def __init__(self, maxDisappeared=50):      # C O N S T R U C T O R:
        self.nextObjectID = 0                   # here the next upcoming object ID is storeded (used in register())
        self.objects = OrderedDict()            # a dictionary is created for all existing objects - ID:[X-pos, Y-Pos]
        self.disappeared = OrderedDict()        # a dictionary is created - info how long a object was invisible ID:frames
        self.maxDisappeared = maxDisappeared
        
    # register a newly found car 
    # centroid - the x / y possition of the objet as an Humpy Array
    def register(self, centroid):                   
        self.objects[self.nextObjectID] = centroid  # create a dictonary entry for the object and store its position
        self.disappeared[self.nextObjectID] = 0     # create a dictonary entry for the objects dissapearance timer and set it to Zero
        self.nextObjectID += 1                      # calculate the next object ID

    # if the car is gone, kick it out
    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    # update each object separatly by stepping through all objects detected in n-dimensional array rects
    def update(self, rects):
        if len(rects) == 0:                                           # if no object was found you can delete them all
            for objectID in list(self.disappeared.keys()):            # for all remaining objects
                self.disappeared[objectID] += 1                       # increment the dissapearance counter
                if self.disappeared[objectID] > self.maxDisappeared:  # if gobe for to long
                    self.deregister(objectID)                         # delete this object
            return self.objects                                       # jump back

        inputCentroids = np.zeros((len(rects), 2), dtype="int")       # create a empty numpy array element for each found objects and its coordinates (reservaze space only)
        for (i, (startX, startY, endX, endY)) in enumerate(rects):    # step through all and map all rects elements to i, startX, startY, endX, endY
            cX = int((startX + endX) / 2.0)                           # calculate the center X position
            cY = int((startY + endY) / 2.0)                           # calculate the center Y position
            inputCentroids[i] = (cX, cY)                              # store the center positions in the numpy array inputCentroids[i]

        if len(self.objects) == 0:                                    # check how many objects are "alive" right now
            for i in range(len(inputCentroids)):                      # looks like none, so create them all
                self.register(inputCentroids[i])                      
        else:                                                         # looks like there is a living object
            objectIDs = list(self.objects.keys())                     # get all current object dicts ID:[x,y]
            objectCentroids = list(self.objects.values())             # get all current object IDs

            D = dist.cdist(np.array(objectCentroids), inputCentroids) # compute the distance between each pair of stored/new centroids
            rows = D.min(axis=1).argsort()                            # find the smallest distance fro each ID inthe newdata
            cols = D.argmin(axis=1)[rows]                             # find the index of the smallest distance for each stored object and store it in cols

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):                        # step through all found rows and columns
                if row in usedRows or col in usedCols:                # if either the row or column is already used, skip it
                    continue
                objectID = objectIDs[row]                             # get the object ID for the current row
                self.objects[objectID] = inputCentroids[col]          # update the ID:[x,y] of the object with the new centroid
                self.disappeared[objectID] = 0                        # remeber that you've seen this object
                usedRows.add(row)                                     # mark the row as used 
                usedCols.add(col)                                     # mark the column as used

            unusedRows = set(range(0, D.shape[0])).difference(usedRows) # find all unused rows - to mark them as unvisible
            unusedCols = set(range(0, D.shape[1])).difference(usedCols) # find all unused columns - to mark them as new objects

            
            for row in unusedRows:                                    # step through all not existing anymore objects
                objectID = objectIDs[row]                             # get the object ID for the current row
                self.disappeared[objectID] += 1                       # increment the dissapearance counter
                if self.disappeared[objectID] > self.maxDisappeared:  # if gone for to long
                    self.deregister(objectID)                         # delete this object

            for col in unusedCols:                                    # step through all new objects
                self.register(inputCentroids[col])                    # register each new object


        return self.objects

# fianlly create the tracker object
tracker = CentroidTracker()

Open the video and store it

In [7]:
# Open video
cap = cv2.VideoCapture(video_path)
print("Opened:", cap.isOpened())

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

Opened: True


create a VideoWriter for output

In [8]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

Now do the magic

In [ ]:
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

#    rects = []
#    for i in range(detections.shape[2]):
#        confidence = detections[0, 0, i, 2]
#        if confidence > 0.4:
#            idx = int(detections[0, 0, i, 1])
#            if CLASSES[idx] in ["car", "bus", "person"]:
#                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
#                rects.append(box.astype("int"))

    CONF_THRESH = 0.55          # etwas höher für stabilere Boxen
    NMS_THRESH  = 0.30          # übliches NMS-IoU

    rects, scores, classes = [], [], []

    for i in range(detections.shape[2]):
        conf = float(detections[0, 0, i, 2])
        if conf >= CONF_THRESH:
            idx = int(detections[0, 0, i, 1])
            label = CLASSES[idx]
            if label in ["car", "bus", "person"]:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])  # [x1,y1,x2,y2]
                rects.append(box.astype(int))
                scores.append(conf)
                classes.append(label)

    # NMS anwenden (nur wenn es überhaupt Boxen gibt)
    if rects:
        keep_idx = nms_boxes(rects, scores, iou_thresh=NMS_THRESH)
        rects   = [rects[i]   for i in keep_idx]
        classes = [classes[i] for i in keep_idx]
    else:
        rects, classes = [], []
        
    objects = tracker.update(rects)

    for (objectID, centroid) in objects.items():
        text = f"ID {objectID}"
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

    out.write(frame)

# Release resources AFTER loop
cap.release()
out.release()

print(f"Frames processed: {frame_count}")
print("Annotated video with persistent IDs saved as output_tracked.mp4")
print("To display in Jupyter, use:")
print("from IPython.display import Video; Video('output_tracked.mp4', embed=True)")

No make it available in Jupiter Labs - need to convert it....

In [ ]:



# JupyterLab video display helpers (Codespaces-safe)
# --------------------------------------------------
# Drop this in a new cell AFTER writing your video file.
# It embeds the MP4 inline (IPython Video widget) and also via raw HTML as fallback.
# If inline playback fails (codec issue), see the optional ffmpeg conversion block below.



VIDEO_IN = output_path
VIDEO_OUT = VIDEO_IN.replace('.mp4', '_h264.mp4')
WIDTH = 800

if not os.path.exists(VIDEO_IN):
    raise FileNotFoundError(f"Input video not found: {VIDEO_IN}")

# 1) Ensure ffmpeg is available via imageio
try:
    import imageio_ffmpeg
except ImportError:
    !pip -q install imageio[ffmpeg]
    import imageio_ffmpeg

ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()
print('Using ffmpeg at:', ffmpeg_path)

# 2) Run conversion to H.264 + yuv420p + faststart
import subprocess
cmd = [
    ffmpeg_path, '-y', '-i', VIDEO_IN,
    '-c:v', 'libx264', '-preset', 'veryfast', '-crf', '23',
    '-pix_fmt', 'yuv420p', '-movflags', '+faststart',
    '-c:a', 'aac', '-b:a', '128k',
    VIDEO_OUT
]
print('Converting ->', VIDEO_OUT)
proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
if proc.returncode != 0:
    print(proc.stdout)
    print(proc.stderr)
    raise RuntimeError('ffmpeg conversion failed')

print('OK:', VIDEO_OUT)

# 3) Embed converted video
try:
    display(Video(VIDEO_OUT, embed=True, width=WIDTH))
except Exception as e:
    print('Embed failed:', e)